In [1]:
import cv2
import dlib
from scipy.spatial import distance 
from imutils import face_utils
import time
import numpy as np
#pip install dlib
#pip install time
#pip install imutils
#pip install cv2
# uncomment above mentioned lines to install necessary libraries

In [2]:
def smiledetector(coordinates):
    # compute the euclidean distances between the 3 sets  mouth landmarks (x, y)-coordinates
    A = distance.euclidean(coordinates[3],coordinates[9])
    B = distance.euclidean(coordinates[2],coordinates[10])
    C = distance.euclidean(coordinates[4],coordinates[8])
    # compute and return the mouth aspect ratio
    average = (A+B+C)/3
    length = distance.euclidean(coordinates[0],coordinates[6])
    return average/length

In [3]:
def eye_aspect_ratio(eye):
    # compute the euclidean distances between the two sets of vertical eye landmarks (x, y)-coordinates
    D = distance.euclidean(eye[1], eye[5])
    E = distance.euclidean(eye[2], eye[4])
    # compute the euclidean distance between the horizontal eye landmark (x, y)-coordinates
    F = distance.euclidean(eye[0], eye[3])
    # compute and return the eye aspect ratio
    ear = (D + E) / (2.0 * F)
    return ear

In [4]:
cap = cv2.VideoCapture(0) #Creating the object, starting video stream
# initialize dlib's face detector (HOG-based) and then create the facial landmark predictor
detect = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")#  download from https://github.com/AKSHAYUBHAT/TensorFace/blob/master/openface/models/dlib/shape_predictor_68_face_landmarks.dat
n=1
# define two constants for the eye aspect ratio to indicate blink
EYE_AR_THRESH = 0.28
# initialize the frame counters and the total number of blinks
COUNTER = 0
# the indexes of the facial landmarks for the left and right eye, respectively
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]


In [5]:
while True:
    ret, img = cap.read()  # Reading images

    if not ret:           #Checking that image is recieved
        break

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    boxes = detect(gray)  # Detection the location of faces
    for i,d in enumerate(boxes):
        shape = predictor(gray, d) #Applying predictor on each face we detect
        shape = face_utils.shape_to_np(shape) # Converting the object to numpy array
        leftEye = shape[lStart:lEnd] # extract the left and right eye coordinates
        rightEye = shape[rStart:rEnd]
        leftEAR = eye_aspect_ratio(leftEye)# then use the coordinates to compute the eye aspect ratio for both eyes
        rightEAR = eye_aspect_ratio(rightEye)
        ear = (leftEAR + rightEAR) / 2.0
        mar = smiledetector(shape[48:68]) # passing coordinates
        status="Smile with your eyes open to take a selfie"
        # checking the smile and open eyes
        if ((mar <0.30 or mar >0.38) and ear > EYE_AR_THRESH): # adjust mar ranges according to you mar values for smile with teeth(<0.3) and withought teeth(>0.38)
            COUNTER += 1
            
        else:
            if COUNTER >= 6:
                print("Captured. Great!")
                status="Captured. Great!"
                cv2.imwrite("Image_"+str(n)+".jpg",img) #saveing the image
                n=n+1
            else:
                print("Smile with your eyes open to take a selfie")
                status="Smile with your eyes open to take a selfie"
            COUNTER = 0 # reset the counter
            
        # printing on video stream window status or mar
        cv2.putText(img, "mar:{}".format(mar), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        #cv2.putText(img, "{}".format(status), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    cv2.imshow("Image_"+str(n)+".jpg", img)    # Showing the video feed
    key = cv2.waitKey(1) & 0xFF  
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break                       

Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Captured. Great!
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a s

Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Captured. Great!
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a selfie
Smile with your eyes open to take a s

In [6]:
cap.release()
cv2.destroyAllWindows()